Задание 1.

Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Функция должна быть написана на python. В ходе решения можно тестировать работу функции как на полном датасете, так и на части (сэмпле) данных.

In [1]:
import pandas as pd

import numpy as np

import seaborn as sns
%matplotlib inline

import matplotlib.pyplot as plt

import datetime

from matplotlib.ticker import FuncFormatter

import matplotlib.ticker as ticker

from scipy import stats

from tqdm.auto import tqdm

from scipy.stats import norm

from scipy.stats import mannwhitneyu

import statsmodels.api as sms

In [2]:
#Загрузим все наши таблицы и посмотрим на их содержание

In [3]:
#данные о времени регистрации
df_reg= pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-trudoljubova/shared/problem1-reg_data.csv', sep=';')
df_reg.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [4]:
#данные о времени захода пользователей в игру
df_auth= pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-trudoljubova/shared/problem1-auth_data.csv', sep= ';')
df_auth.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [5]:
#Напишем функцию для объединения датафреймов в общий 

def df_consolidation():
    
    df_reg.reg_ts = df_reg.reg_ts.apply(lambda x: datetime.datetime.fromtimestamp(x).date())
    df_auth.auth_ts = df_auth.auth_ts.apply(lambda x: datetime.datetime.fromtimestamp(x).date())
  
    ar_df = df_auth.merge(df_reg, how='left', on='uid')
    
    return ar_df

In [6]:
ar_df = df_consolidation()

In [7]:
#Посмотрим на наши данные
ar_df.head()

,auth_ts,uid,reg_ts
0,1998-11-18,1,1998-11-18
1,1999-07-23,2,1999-07-23
2,1999-07-25,2,1999-07-23
3,1999-07-31,2,1999-07-23
4,1999-08-05,2,1999-07-23


In [8]:
ar_df.dtypes

auth_ts    object
uid         int64
reg_ts     object
dtype: object

In [9]:
ar_df.describe()

,uid
count,9.601013e+06
mean,3.347198e+05
std,2.832669e+05
min,1.000000e+00
25%,9.599700e+04
50%,2.579210e+05
75%,5.190250e+05
max,1.110622e+06


In [10]:
#Напишем функцию для расчета retention, где 
# df - данные для расчёта, 
# Nlast_days - количество последних дней, за которые мы посмотрим retention (для наглядности возьмем 30 дней)

In [11]:
def retention(df, Nlast_days=30):

    last_day = df.reg_ts.max() #Определим последний день в данных
    start_day = last_day - pd.offsets.Day(Nlast_days) #Определим день, с которого начнем отсчет retention
    
#Обработаем наши данные для работы с когортами и создадим Pivot table, где дни-когорты будут индексы, а n_day в колонках.
    cohorts = df.query('reg_ts > @start_day').rename(columns={'reg_ts': 'reg_day_', 'auth_ts': 'auth_day'})
    cohorts['n_day'] = (cohorts['auth_day'] - cohorts['reg_day_']).dt.days
    cohorts['reg_day'] = cohorts['reg_day_'].apply(lambda x: x.strftime('%Y-%m-%d'))
    cohorts = cohorts\
                        .groupby(['reg_day', 'auth_day', 'n_day'], as_index=False)\
                        .agg({'uid': pd.Series.nunique})\
                        .rename(columns={'uid': 'total_users'})\
                        .pivot(index='reg_day', columns='n_day', values='total_users')

    cohort_group_size = cohorts[0] #Определяем размеры каждой когорты

    user_retention = cohorts.divide(cohort_group_size, axis=0) #Разделим каждодневное количество пользователей для ккаждой когорты на размер когорты

    
#Теперь создадим функцию для нашей таблицы retention
    def make_style(df):
        return (df
                    .style
                    .set_caption('User retention cohort')
                    .set_table_styles([{
                                    'selector': 'caption',
                                    'props': [
                                        ('color', 'black'),
                                        ('font-size', '25px'),
                                        ('text-align','center')
                                    ]
                                }])) #Подпись таблицы
                
    return make_style(user_retention)

In [12]:
#Построим таблицу для нашего retention
retention(ar_df, 18)

n_day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
reg_day,,,,,,,,,,,,,,,,,,
2020-09-06,1.000000,0.035044,0.043179,0.051314,0.058198,0.058198,0.069462,0.064456,0.056320,0.055695,0.055069,0.055695,0.046308,0.051314,0.058198,0.050688,0.050688,0.031289
2020-09-07,1.000000,0.027483,0.041224,0.054966,0.049969,0.066833,0.063710,0.057464,0.041849,0.046846,0.056215,0.059963,0.051218,0.043098,0.045597,0.047470,0.039975,nan
2020-09-08,1.000000,0.019327,0.035536,0.034289,0.058603,0.070449,0.072319,0.066708,0.049875,0.056110,0.062344,0.056110,0.052369,0.051746,0.052993,0.036783,nan,nan
2020-09-09,1.000000,0.021171,0.033624,0.044832,0.062889,0.067870,0.059153,0.060399,0.045455,0.054795,0.037983,0.056040,0.047323,0.044209,0.028643,nan,nan,nan
2020-09-10,1.000000,0.018024,0.044127,0.045991,0.043505,0.069608,0.057800,0.068365,0.049720,0.041019,0.060907,0.047856,0.052206,0.036669,nan,nan,nan,nan
2020-09-11,1.000000,0.014277,0.039727,0.039106,0.061453,0.063315,0.068901,0.062694,0.049038,0.039106,0.055866,0.061453,0.039106,nan,nan,nan,nan,nan
2020-09-12,1.000000,0.017968,0.040892,0.039653,0.040273,0.063197,0.068773,0.057621,0.046468,0.051425,0.052045,0.029740,nan,nan,nan,nan,nan,nan
2020-09-13,1.000000,0.022263,0.044527,0.039579,0.050093,0.061843,0.066172,0.051948,0.048237,0.050093,0.044527,nan,nan,nan,nan,nan,nan,nan
2020-09-14,1.000000,0.019753,0.038889,0.042593,0.058025,0.062963,0.068519,0.068519,0.044444,0.033951,nan,nan,nan,nan,nan,nan,nan,nan


In [13]:
#Для удобства переведем значения в проценты и раскрасим в более красивую таблицу:)
retention(ar_df, 18).format("{:.2%}", na_rep="").background_gradient(cmap='PuBu')

n_day,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
reg_day,,,,,,,,,,,,,,,,,,
2020-09-06,100.00%,3.50%,4.32%,5.13%,5.82%,5.82%,6.95%,6.45%,5.63%,5.57%,5.51%,5.57%,4.63%,5.13%,5.82%,5.07%,5.07%,3.13%
2020-09-07,100.00%,2.75%,4.12%,5.50%,5.00%,6.68%,6.37%,5.75%,4.18%,4.68%,5.62%,6.00%,5.12%,4.31%,4.56%,4.75%,4.00%,
2020-09-08,100.00%,1.93%,3.55%,3.43%,5.86%,7.04%,7.23%,6.67%,4.99%,5.61%,6.23%,5.61%,5.24%,5.17%,5.30%,3.68%,,
2020-09-09,100.00%,2.12%,3.36%,4.48%,6.29%,6.79%,5.92%,6.04%,4.55%,5.48%,3.80%,5.60%,4.73%,4.42%,2.86%,,,
2020-09-10,100.00%,1.80%,4.41%,4.60%,4.35%,6.96%,5.78%,6.84%,4.97%,4.10%,6.09%,4.79%,5.22%,3.67%,,,,
2020-09-11,100.00%,1.43%,3.97%,3.91%,6.15%,6.33%,6.89%,6.27%,4.90%,3.91%,5.59%,6.15%,3.91%,,,,,
2020-09-12,100.00%,1.80%,4.09%,3.97%,4.03%,6.32%,6.88%,5.76%,4.65%,5.14%,5.20%,2.97%,,,,,,
2020-09-13,100.00%,2.23%,4.45%,3.96%,5.01%,6.18%,6.62%,5.19%,4.82%,5.01%,4.45%,,,,,,,
2020-09-14,100.00%,1.98%,3.89%,4.26%,5.80%,6.30%,6.85%,6.85%,4.44%,3.40%,,,,,,,,
